In [ ]:
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
from pathlib import Path
import matplotlib.pylab as plt
import numpy as np

import pism_ragis.interpolation

xr.set_options(keep_attrs=True)


In [ ]:
def extend_domain(da: xr.DataArray, n) -> xr.DataArray:

    dx = float(da.x[1] - da.x[0])
    x0 = float(da.x[0])
    x_last = float(da.x[-1])
    new_x = xr.DataArray(
        np.arange(x0 - dx * (n + 1), x_last + dx * (n + 1), dx),
        dims="x",
        name="x",
    )
    dy = float(da.y[1] - da.y[0])
    y0 = float(da.y[0])
    y_last = float(da.y[-1])
    new_y = xr.DataArray(
        np.arange(y0 - dy * (n + 1), y_last  + dy * (n + 1), dy),
        dims="y",
        name="y",
    )
    da_extended = da.reindex(x=new_x, y=new_y)

    return da_extended

In [ ]:
bm = xr.open_dataset("dem/BedMachineGreenland-v5_0.nc")
bm = bm.sortby("y")
arctic_dem = rxr.open_rasterio("dem/arctic-dem-qaamerujup_0.tif").squeeze(drop=True)
arctic_dem = arctic_dem.where(arctic_dem > 0.1, 0)
q_dem = rxr.open_rasterio("dem/1931_DEMrek_TIN10_clip.tif").squeeze(drop=True)
q_dem = extend_domain(q_dem, 10)
q_dem = q_dem.where(q_dem>0)

q_arctic_dem = q_dem.interp_like(arctic_dem)

gdf = gpd.read_file("dem/1931-dem-buffered.gpkg").to_crs("EPSG:3413")
exclude = gpd.read_file("dem/qaamerujup-exclude.gpkg").to_crs("EPSG:3413")

In [ ]:
arctic_q_dem  = arctic_dem.interp_like(q_dem).rio.clip(gdf.geometry, drop=False, invert=True, all_touched=True)
arctic_q_dem = arctic_q_dem.where(arctic_q_dem>0.1, np.nan)

a = arctic_q_dem
b = q_dem

both_valid = a.notnull() & b.notnull()
a_valid    = a.notnull() & b.isnull()
b_valid    = b.notnull() & a.isnull()

c = xr.where(
    both_valid,
    a,                
    xr.where(a_valid, a, xr.where(b_valid, b, xr.full_like(a, float("nan")))),
)
c = c.utils.fillna()

q_dem_arctic_domain = c.interp_like(arctic_dem)
dem_merged = xr.where(q_dem_arctic_domain.notnull(), q_dem_arctic_domain, arctic_dem)
dem_merged.name = "surface"
if hasattr(dem_merged.spatial_ref, "GeoTransform"):
    del dem_merged.spatial_ref.attrs["GeoTransform"]
    
dem_merged.rio.write_crs("EPSG:3413", inplace=True)
dem_merged.to_netcdf("c.nc")

In [ ]:
x_attrs = bm.x.attrs
y_attrs = bm.y.attrs
surface_attrs = bm.surface.attrs
thickness_attrs = bm.thickness.attrs
mask_attrs = bm.mask.attrs

bm_q = bm.interp_like(dem_merged)
bm_q["surface"].values = dem_merged.values
bm_q["thickness"] = xr.where(bm_q["surface"] > 0.1, bm_q["surface"] - bm_q["bed"], 0)
bm_q["thickness"] = bm_q["thickness"].where(bm_q["thickness"] > 0, 0)
exclude_q = bm_q["thickness"].rio.write_crs("EPSG:3413", inplace=True).rio.clip(exclude.geometry, drop=False, invert=True, all_touched=True).drop_vars("mapping")
bm_q["thickness"] = bm_q["thickness"].where(exclude_q.notnull(), 0)
bm_q["surface"] = bm_q["surface"].where(exclude_q.notnull(), 0)
bm_q["bed"] = bm_q["bed"].where(exclude_q.notnull(), -1000)
bm_q["mask"] = bm_q["mask"].where(exclude_q.notnull(), 0)

bm_q["x"].attrs.clear()
bm_q["x"].attrs.update(x_attrs)
bm_q["y"].attrs.clear()
bm_q["y"].attrs.update(y_attrs)
bm_q["surface"].attrs.clear()
bm_q["surface"].attrs.update(surface_attrs)
bm_q["thickness"].attrs.clear()
bm_q["thickness"].attrs.update(thickness_attrs)
bm_q["mask"].attrs.clear()
bm_q["mask"].attrs.update(mask_attrs)
bm_q = bm_q.fillna(0)
for name in bm_q.data_vars:
    if name in bm_q:
        bm_q[name].encoding.update({"_FillValue": None, "zlib": True, "complevel": 2})

bm_q.to_netcdf("dem/bedmachine-1931-qaamerujup.nc")

In [ ]:
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
ds = xr.open_dataset("/Users/andy/base/pism-ragis/data/climate/MARv3.5.2-20CRv2c_1900_2014.nc", decode_times=time_coder, decode_timedelta=True)

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import re

fn = "/Users/andy/base/pism-ragis/data/climate/MARv3.5.2-20CRv2c_1900_2014.nc"

# 1. Open *without* CF time decoding
ds_raw = xr.open_dataset(fn, decode_times=False)

time_var = ds_raw["time"]
units = time_var.attrs.get("units", "")
calendar = time_var.attrs.get("calendar", "standard")
print(units, calendar)
# -> "months since 1900-1-1 00:00:00", "standard"

# 2. Parse the reference date from the units string
m = re.match(r"months since (\d+)-(\d+)-(\d+)", units)
if m is None:
    raise ValueError(f"Unexpected time units: {units!r}")
year0, month0, day0 = map(int, m.groups())
base = pd.Timestamp(year0, month0, day0)

# 3. Convert integer month offsets to real datetimes
offset_months = time_var.values.astype(int)  # e.g. 0, 1, 2, ...
new_time = pd.to_datetime(
    [base + pd.DateOffset(months=int(k)) for k in offset_months]
)

# 4. Attach the fixed time coordinate to a new dataset
ds = ds_raw.assign_coords(time=("time", new_time))

# Optionally, clean CF-ish metadata for time
ds["time"].attrs.update(
    {
        "standard_name": "time",
        "long_name": "time",
        "calendar": "proleptic_gregorian",
        "units": "days since 1900-01-01 00:00:00",
    }
)

ds


In [ ]:
ds = ds.drop_vars("time_bnds", errors="ignore").rio.set_spatial_dims(x_dim="x", y_dim="y")
ds.rio.write_crs("EPSG:3413", inplace=True)
ds_clipped = ds.rio.clip(gdf.geometry, drop=True, invert=False, all_touched=True)
sums = ds_clipped.mean(dim=["x", "y"])

In [ ]:
sums.climatic_mass_balance.plot()

In [ ]:

month_length = sums.time.dt.days_in_month
weights = month_length.groupby("time.month") / month_length.groupby("time.month").sum()
sums_weighted = (sums * weights).groupby("time.month").sum(dim="time").rename({"month": "time"})


In [ ]:
sums_weighted

In [ ]:
ds = xr.open_dataset("/Users/andy/base/pism